Here, we look at a new dataset and try to answer some questions about it.

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/iowa_liquor/Iowa_Liquor_Sales.csv', index_col=0)

In [3]:
df.head()

,Invoice/Item Number,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,...,Item Number,Item Description,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons)
0,S29198800001,11/20/2015,2191,Keokuk Spirits,1013 MAIN,KEOKUK,52632,"1013 MAIN\nKEOKUK 52632\n(40.39978, -91.387531)",56.0,Lee,...,297,Templeton Rye w/Flask,6,750,$18.09,$27.14,6,$162.84,4.50,1.19
1,S29195400002,11/21/2015,2205,Ding's Honk And Holler,900 E WASHINGTON,CLARINDA,51632,"900 E WASHINGTON\nCLARINDA 51632\n(40.739238, ...",73.0,Page,...,297,Templeton Rye w/Flask,6,750,$18.09,$27.14,12,$325.68,9.00,2.38
2,S29050300001,11/16/2015,3549,Quicker Liquor Store,1414 48TH ST,FORT MADISON,52627,"1414 48TH ST\nFORT MADISON 52627\n(40.624226, ...",56.0,Lee,...,249,Disaronno Amaretto Cavalli Mignon 3-50ml Pack,20,150,$6.40,$9.60,2,$19.20,0.30,0.08
3,S28867700001,11/04/2015,2513,Hy-Vee Food Store #2 / Iowa City,812 S 1ST AVE,IOWA CITY,52240,812 S 1ST AVE\nIOWA CITY 52240\n,52.0,Johnson,...,237,Knob Creek w/ Crystal Decanter,3,1750,$35.55,$53.34,3,$160.02,5.25,1.39
4,S29050800001,11/17/2015,3942,Twin Town Liquor,104 HIGHWAY 30 WEST,TOLEDO,52342,"104 HIGHWAY 30 WEST\nTOLEDO 52342\n(41.985887,...",86.0,Tama,...,249,Disaronno Amaretto Cavalli Mignon 3-50ml Pack,20,150,$6.40,$9.60,2,$19.20,0.30,0.08


In [4]:
df.columns

Index(['Invoice/Item Number', 'Date', 'Store Number', 'Store Name', 'Address',
       'City', 'Zip Code', 'Store Location', 'County Number', 'County',
       'Category', 'Category Name', 'Vendor Number', 'Vendor Name',
       'Item Number', 'Item Description', 'Pack', 'Bottle Volume (ml)',
       'State Bottle Cost', 'State Bottle Retail', 'Bottles Sold',
       'Sale (Dollars)', 'Volume Sold (Liters)', 'Volume Sold (Gallons)'],
      dtype='object')

1) Which store had the highest total dollars of sales?
    - Is this the same store that had the highest volume (in liters) of alcohol sold?

In [62]:
#df.sort_values(by='Sale (Dollars)', ascending=False)
print(f'Store number with highest sales in Dollars: {df.loc[df.sort_values(by="Sale (Dollars)", ascending=False).index[0], "Store Number"]}')
print(f'Store number with highest sales in volume: {df.loc[df.sort_values(by="Volume Sold (Liters)", ascending=False).index[0], "Store Number"]}')

Store number with highest sales in Dollars: 4483.0
Store number with highest sales in volume: 3385.0


2) Which item has the highest cost per liter?

In [120]:
df = df.replace({'\$': ''}, regex = True)
df['State Bottle Cost'] = df['State Bottle Cost'].astype('float')
df['State Bottle Retail'] = df['State Bottle Retail'].astype('float')
df['Sale (Dollars)'] = df['Sale (Dollars)'].astype('float')

def helper_cost_per_liter(row):
    cost_usd = float(row['State Bottle Cost'])#.replace('$', '').replace(',', ''))
    volume_ml = float(row['Bottle Volume (ml)'])
    cost_per_liter = (cost_usd / volume_ml) * 1000
    return cost_per_liter

df['Cost_Per_Liter_USD'] = df.apply(helper_cost_per_liter, axis=1)
print(f'Item number with highest cost per liter: {df.loc[df.sort_values(by="Cost_Per_Liter_USD", ascending=False).index[0], "Item Number"]}')

Item number with highest cost per liter: 905502.0


3) What is the most common pack size?

In [64]:
print(f'Most common pack size: {df["Pack"].value_counts().index[0]}')

Most common pack size: 12.0


4) What is the most commonly sold item?

In [65]:
print(f'Most commonly sold item: {df.loc[df.sort_values(by="Bottles Sold", ascending=False).index[0], "Item Number"]}')

Most commonly sold item: 43337.0


5) What day in 2015 was the most money made? (Across all stores)

In [105]:
df['Date'] = pd.to_datetime(df['Date'])
date = df.set_index('Date')
date.groupby([date.index.year, date.index.month, date.index.day]).sum()['Sale (Dollars)'].idxmax()

(2015.0, 3.0, 16.0)

6) Which zip code drinks the most? (in liters)

In [14]:
df.loc[:, 'Volume_Consumed'] = df.loc[:, 'Bottles Sold'] * df.loc[:, 'Bottle Volume (ml)']
print(f'Zip Code that drinks the most: {df.loc[df.sort_values(by="Volume_Consumed", ascending=False).index[0], "Zip Code"]}')
      

Zip Code that drinks the most: 52402


7) Does the data set contain any null values?

In [9]:
df.isnull().any().any()

True

8) What is the average state bottle cost?
    - Max cost?
    - Min cost?
    - 50%?

In [10]:
df['State Bottle Cost'].mean()
#df['State Bottle Cost'].min()
#df['State Bottle Cost'].max()
#df['State Bottle Cost'].median()

9.4551807

9) What is the total volume sold in Iowa City in 2013?

In [141]:
i_c_2013 = df.groupby([date.index.year, 'City']).sum()
i_c_2013.loc[(slice(2013,2013), slice('IOWA CITY','IOWA CITY')), 'Volume Sold (Liters)']

Date    City     
2013.0  IOWA CITY    9063.01
Name: Volume Sold (Liters), dtype: float64

10) Which month (average, across all years) sees the highest sales?

In [145]:
df.groupby([date.index.month]).sum().sort_values(by='Sale (Dollars)', ascending=False).index[0]

10.0

11) Which item has the lowest cost per bottle in a pack?

In [41]:
df2 = df[df['State Bottle Cost'] >= 0.01]
df2['Cost_per_bottle'] = df2.loc[:, 'State Bottle Cost'] / df2.loc[:, 'Pack']
df2.groupby('Item Description').min()['Cost_per_bottle'].sort_values(ascending = True).index[0]

/opt/conda/envs/dsr/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


'Fireball Cinnamon Whisky'

12) What is the most popular item at Wilkie Liquors?

In [119]:
df.iloc[df[df['Store Name']=='Wilkie Liquors'].sort_values(by='Bottles Sold', ascending=False).index[0]]

Invoice/Item Number                                           S27711800084
Date                                                   2015-09-03 00:00:00
Store Number                                                          5102
Store Name                                                  Wilkie Liquors
Address                                                   724, 1st  ST  SE
City                                                          MOUNT VERNON
Zip Code                                                             52314
Store Location           724, 1st ST SE\nMOUNT VERNON 52314\n(41.91776,...
County Number                                                           57
County                                                                Linn
Category                                                        1.0817e+06
Category Name                                  DISTILLED SPIRITS SPECIALTY
Vendor Number                                                          434
Vendor Name              

13) From which item does Wilkie Liquors earn the most of its revenue?

In [124]:
df.iloc[df[df['Store Name']=='Wilkie Liquors'].sort_values(by='Sale (Dollars)', ascending=False).index[0]]

Invoice/Item Number                                           S27711800084
Date                                                   2015-09-03 00:00:00
Store Number                                                          5102
Store Name                                                  Wilkie Liquors
Address                                                   724, 1st  ST  SE
City                                                          MOUNT VERNON
Zip Code                                                             52314
Store Location           724, 1st ST SE\nMOUNT VERNON 52314\n(41.91776,...
County Number                                                           57
County                                                                Linn
Category                                                        1.0817e+06
Category Name                                  DISTILLED SPIRITS SPECIALTY
Vendor Number                                                          434
Vendor Name              

14) Do prices for the same item differ across different stores?

In [55]:
df[df['Item Description'] == 'Captain Morgan Spiced Rum'][['Store Name', 'State Bottle Cost']].head()

,Store Name,State Bottle Cost
124,"Brothers Market, Inc.",17.75
129,Geno's Liquor,11.30
238,Hy-Vee Food Store / Cedar Falls,10.99
364,Pit Stop Liquor and Tobacco,5.00
476,LIQUOR AND GROCERY DEPOT,5.00


15) Which store sells the most liquor in the city of Des Moines?

In [126]:
df.columns

Index(['Invoice/Item Number', 'Date', 'Store Number', 'Store Name', 'Address',
       'City', 'Zip Code', 'Store Location', 'County Number', 'County',
       'Category', 'Category Name', 'Vendor Number', 'Vendor Name',
       'Item Number', 'Item Description', 'Pack', 'Bottle Volume (ml)',
       'State Bottle Cost', 'State Bottle Retail', 'Bottles Sold',
       'Sale (Dollars)', 'Volume Sold (Liters)', 'Volume Sold (Gallons)',
       'Cost_Per_Liter_USD', 'Volume_Consumed', 'Cost_per_bottle'],
      dtype='object')

In [137]:
cities = df.groupby(['City', 'Store Name']).sum()
cities.loc['DES MOINES', 'Volume Sold (Liters)'].sort_values(ascending=False).head(1)

Store Name
Hy-Vee #3 / BDI / Des Moines    24248.25
Name: Volume Sold (Liters), dtype: float64

16) Make a plot about at least one of these questions: